Define a set of noise-contrastive learning paradigms which instantiate different types of word form codes.
Concretely, the negative sampling space defines the contrast set of relevant alternatives. A more modular word form code would sample broadly from the entire lexicon; a less modular word-form code would take into account contextual constraints that rule out many words in the lexicon.

These NCE objectives should impose constraints on the resulting optimal code.